In [12]:
from typing import Dict

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


from typing import Dict

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_tabular_classification_sample(
    project: str,
    endpoint_id: str,
    instance_dict: Dict,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # for more info on the instance schema, please use get_model_sample.py
    # and look at the yaml found in instance_schema_uri
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # See gs://google-cloud-aiplatform/schema/predict/prediction/tabular_classification_1.0.0.yaml for the format of the predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

In [2]:
import os
# gcsfuse로 버킷 마운트 하기
BUCKET_NAME='bucket-titanic-dw-202311280412-test-gogogogo1234-auto-ml'
MOUNT_PATH ='/home/jupyter/gcs2' 
os.makedirs(MOUNT_PATH, exist_ok=True)
!gcsfuse --implicit-dirs {BUCKET_NAME} {MOUNT_PATH}

{"time":"29/11/2023 12:27:55.758276","severity":"INFO","msg":"Start gcsfuse/1.2.0 (Go version go1.21.0) for app \"\" using mount point: /home/jupyter/gcs2\n"}


In [9]:
import pandas as pd
df = pd.read_csv('/home/jupyter/gcs2/titanic_test_dw.csv')
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Age_Cat,FamilySize,Survived
0,3,1,1.0,0,0,7.7500,1,5,1,0
1,2,1,1.0,0,0,10.5000,2,5,1,0
2,3,0,1.0,1,2,23.4500,2,5,4,0
3,3,0,1.0,0,0,7.8792,1,5,1,1
4,3,1,1.0,0,0,8.0500,2,5,1,0


In [5]:
#test_label = df['Survived']

In [6]:
# df.drop('Survived', axis=1, inplace=True)
# df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Age_Cat,FamilySize
0,3,1,1.0,0,0,7.7500,1,5,1
1,2,1,1.0,0,0,10.5000,2,5,1
2,3,0,1.0,1,2,23.4500,2,5,4
3,3,0,1.0,0,0,7.8792,1,5,1
4,3,1,1.0,0,0,8.0500,2,5,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      90 non-null     int64  
 1   Sex         90 non-null     int64  
 2   Age         90 non-null     float64
 3   SibSp       90 non-null     int64  
 4   Parch       90 non-null     int64  
 5   Fare        90 non-null     float64
 6   Embarked    90 non-null     int64  
 7   Age_Cat     90 non-null     int64  
 8   FamilySize  90 non-null     int64  
 9   Survived    90 non-null     int64  
dtypes: float64(2), int64(8)
memory usage: 7.2 KB


In [13]:
# for i in range(len(df)):
i=1
INSTANCE = df.drop('Survived', axis=1,).to_dict(orient='records')[i]
    


In [14]:
INSTANCE

{'Pclass': 2,
 'Sex': 1,
 'Age': 1.0,
 'SibSp': 0,
 'Parch': 0,
 'Fare': 10.5,
 'Embarked': 2,
 'Age_Cat': 5,
 'FamilySize': 1}

In [19]:
# INSTANCE = {
#     "petal_length": "1.4",
#     "petal_width": "1.3",
#     "sepal_length": "5.1",
#     "sepal_width": "2.8",
# }

INSTANCE = {'Pclass': '2',
 'Sex': '1',
 'Age': '1.0',
 'SibSp': '0',
 'Parch': '0',
 'Fare': '10.5',
 'Embarked': '2',
 'Age_Cat': '5',
 'FamilySize': '1'}
    
    
predict_tabular_classification_sample(
    project="226735916567",
    endpoint_id="3814392753731665920",
    location="us-central1",
    instance_dict=INSTANCE,
    api_endpoint="us-central1-aiplatform.googleapis.com"
)

response
 deployed_model_id: 3871272689259773952
 prediction: {'classes': ['0', '1'], 'scores': [0.8496460914611816, 0.1503539532423019]}
